In [2]:
# imports für Tesseract
from PIL import Image, ImageOps
import pytesseract

# imports für OpenAI
import openai
import os
import base64
from dotenv import load_dotenv

# Transkription von Maschinen- und Handschrift

Maschinenschrifterkennung (optical character recognition, OCR) kann mit Tesseract umgesetzt werden, einer bereits seit den 80ern in Entwicklung befindlicher Software, die inzwischen Open Source und quasi-Standard in diesem Bereich ist. Um Tesseract mit Python verwenden zu können, muss die Anwendung zunächst installiert werden, s.: https://tesseract-ocr.github.io/tessdoc/Installation.html Unter Windows ist darauf zu achten, dass die CL-Anwendung auf dem Pfad liegen mussen (s. Umgebungsvariablen). 

In [11]:
def preprocess_image(image_path, threshold=150):
    """
    Vorverarbeitung des Bildes, um den Kontrast vom Text zum Hintergrund zu schärfen. 
    """
    image = Image.open(image_path).convert("L")  # "L" = 8-bit grayscale
    return image.point(lambda x: 0 if x < threshold else 255, '1')

def ocr_image(image_path, lang="deu"):
    image = preprocess_image(image_path)
    return pytesseract.image_to_string(image, lang=lang)

ocr_image("img/letter_gertrud_1.png")

'Kulturwisgenschaftiche Biolothek Warburg\n\nHamburg 20\n\nrt seite.\n\n'

Wie deutlich zu erkennen ist, ist das Ergebnis der Transkription sehr schlecht: Für Tesseract ist die Qualität des Ausgangsdokuments nicht gut genug. 

Multimodale LLMs kommen auch mit ungewöhnlichen Dokumenten und einer schlechten Auflösung, und sogar mit Handschriften erstaunlich gut klar. Im folgenden wird die OpenAI-API mit dem o4-Modell verwendet. Dazu ist ein OpenAI API-Key notwendig, der sich über das Entwickler-Dashboard erstellen lässt. 

In [5]:
# API key laden
load_dotenv(".env")
openai.api_key = os.getenv("OPENAI_KEY") 

In [8]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")
    
def transcribe(image_path):
    base64image = encode_image(image_path)
    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "Du bist ein hilfreicher Assistent, der Handschrift transkribiert."},
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "Bitte transkribiere den handschriftlichen Text in diesem Bild so genau wie möglich."},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/png;base64,{base64image}"
                        }
                    }
                ]
            }
        ],
        max_tokens=2000
    )
    return response['choices'][0]['message']['content']

In [10]:
print(transcribe("img/letter_lotte.png"))

London. 12. 12. 1946

My dear Raymond,

Thank you very much for your two letters. One from London and the other from Montreal. How very relieved to see from the cable, which says Aunt sent me, that gift had arrived safely and that letter had got through too, it was not too much of an ordeal. But John's several extra stops in London must have been something of a trial.

Mrs. Hamilton discoursed your last visit very vividly to me. She was full of self-reproach because she let you go to Canada without a proper breakfast and with not seeing you gone coat button on and I had to promise that I would write her apologies to you. There is another woman whose protective and maternal instincts you have thoroughly aroused. I would have written to you earlier but according to my very unheroic fashion I went down with a small attack of the lovely after you went.

Not, however, before I had done some tidying in your room, taking away the things destined for me, gas books, to the Taylor, calling on Hu